In [1]:
import torch
from torchdrug import datasets, tasks, core, models

dataset = datasets.ClinTox("~/molecule-datasets/")

Constructing molecules from SMILES: 100%|██████████| 1484/1484 [00:02<00:00, 604.78it/s]


In [2]:
lengths = [int(0.8 * len(dataset)), int(0.1 * len(dataset))]
lengths += [len(dataset) - sum(lengths)]
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, lengths)

In [4]:
from gatv2 import GATV2

In [5]:
model = GATV2(input_dim=dataset.node_feature_dim, hidden_dims=[64, 64, 64], edge_input_dim=dataset.edge_feature_dim,
                short_cut=True, batch_norm=True, concat_hidden=True)
task = tasks.PropertyPrediction(model, task=dataset.tasks,
                            criterion="bce", metric=("auprc", "auroc"))
optimizer = torch.optim.Adam(task.parameters(), lr=1e-3)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer, batch_size=1024)
solver.train(num_epoch=1)

17:51:22   Preprocess training set
17:51:22   {'batch_size': 1024,
 'class': 'core.Engine',
 'gpus': None,
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.PropertyPrediction',
          'criterion': 'bce',
          'graph_construction_model': None,
          'metric': ('auprc', 'auroc'),
          'mlp_batch_norm': False,
          'mlp_dropout': 0,
          'model': {'activation': 'relu',
                    'batch_norm': True,
                    'class': 'GATV2',
                    'concat_hidden': True,
                    'edge_input_dim': 18,
                    'hidden_dims': [64, 64, 64],
                    'input_dim': 67,
                    'negative_slope': 0.2,
                    'num_he